In [2]:
import pandas as pd
import os
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=UserWarning)
print(os.getcwd())

/Users/nikkivanhandel/ampf


In [3]:
xl = pd.ExcelFile('BuildDocumentation.xlsx')
sheets = xl.sheet_names  # see all sheet names

In [41]:
# Parse the sheet into a nice row 
def sheet_to_row(df,sheet):
    g = 'General Information' # columns with trait names
    # input: preparsed dataframe
    # output: pandas dataframe with traits as columns
    info = df.iloc[:, 0:2]
    info.dropna(axis=0, subset=g, inplace=True)
    row = info.set_index(g).transpose()
    row.loc[:,'Build ID:'] = sheet # Reassign BuildID 
    row.reset_index(drop=True, inplace=True)
    
    # Find where build description is (column changes )
    notecol = np.where(df.iloc[0, :].str.contains('Build Description')==True)[0][0]
    buildinfo = df.iloc[:, notecol:notecol+2].dropna().transpose()
    buildinfo.columns = buildinfo.iloc[0]
    buildinfo = buildinfo.drop(buildinfo.index[0]).reset_index(drop=True)
    if not buildinfo.empty: # If notes, add notes
        row = pd.concat([row, buildinfo], axis=1)
    return row


master = sheet_to_row(xl.parse(sheets[2]), sheets[2])

notetypes = set()

for sheet in sheets[3:]:
    df = xl.parse(sheet)
    row = sheet_to_row(df, sheet)

    master = pd.merge(master, row, how='outer')

    
master.set_index('Build ID:', inplace=True)

# Wrangle datatypes
master = master.infer_objects()

# Remove Empty Columns
for col in master.columns:
    if (~master[col].isna()).sum() < 3: # Remove columns with few observations
        print('Removed ', col)
        master.drop(col, axis=1, inplace=True, errors='ignore')


Removed  Details
Removed  Cost:
Removed  Heat Treat
Removed  Translation Discrepancy
Removed  BP Corner
Removed  Part Corner
Removed  Diff (mm)
Removed  Real (mm)
Removed  9.379999999999995
Removed  2.8424242424242414


In [42]:
def multiremove(string, replaces):
    for word in replaces:
        string = string.replace(word, '')
    return string
    
# Programmatically refactor column names 

cols = [multiremove(col, ['-',':', '?', '_', '&']) for col in master.columns]
for i, col in enumerate(cols):
    if '(' in col:
        p = col[(col.find('(')):(col.find(')')+1)]
        if len(p)==3:
            cols[i]= multiremove(col, ['(', ')']) # For XY
        else:
            cols[i]=col.replace(p, '')
cols = [c.strip().lower().replace(' ', '_') for c in cols]
master.columns=cols

master.successful = master.successful.str.lower().str.contains('yes')

In [46]:
master.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72 entries, B001 to B096
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   nickname                        72 non-null     object        
 1   operator                        72 non-null     object        
 2   date_printed                    70 non-null     datetime64[ns]
 3   customer                        52 non-null     object        
 4   build_plate_type                72 non-null     object        
 5   build_plate_id                  37 non-null     object        
 6   material_type                   70 non-null     object        
 7   file_location                   67 non-null     object        
 8   parameter_file_name             67 non-null     object        
 9   successful                      30 non-null     object        
 10  total_part_volume               35 non-null     object        
 11  print_ti

### Observations
 - Many builds are the same part/project - linked by title
 -  <del>Operators should be comma seperated for consistency  </del>
 - Customer looks fine, but some names are common 
 - <del> BuildPlateType has redundancy. </del>
 - <del>TotalPartVolume, PrintType, GasFlowVoltage must be forced to numeric</del>
 -  <del>Some overlap with powdertype and material  </del>
 - Parameter file names are usually not given
 - Successful Needs consistency 
 -  <del>Heat treat is useless lol  </del>

In [22]:
# Fix formats
for numCol in ['TotalPartVolume', 'PrintTime', 'GasFlowVoltage']:
    master.loc[:, numCol] = pd.to_numeric(master[numCol], errors='coerce')


KeyError: 'TotalPartVolume'

In [ ]:
# Cleanup dataframe 
target = 
#Make operators comma-seperated
master.Operator = master.Operator.str.replace('/', ',')

br, ar = 'Before Replacement: ', 'After Replacement: '

print('BuildPlateTypes')
print(br, master.BuildPlateType.unique())
BP_replace = master.BuildPlateType.str.contains('Full')
master.loc[BP_replace, 'BuildPlateType'] = 'Full'
print(ar, master.BuildPlateType.unique())

BuildPlateTypes


AttributeError: 'DataFrame' object has no attribute 'BuildPlateType'

In [51]:
master.MaterialType.compare(master.PowderType)

,self,other
Build ID:,,
B001,EOS 316L,NaN
B002,EOS 316L,NaN
B003,EOS 316L,NaN
B004,EOS 316L,NaN
B005,EOS 316L,NaN
B006,EOS 316L,NaN
B007,EOS 316L,NaN
B008,EOS 316L,NaN
B009,EOS 316L,NaN


In [52]:
# Inform material from Powder Type
material_df = master.loc[:, ['MaterialType','PowderType']]
unknown_material = material_df.MaterialType.isna() & ~material_df.PowderType.isna()
master.loc[unknown_material, 'MaterialType'] = '316L'

# Presumably EOS branded powder?
eos = master.MaterialType.str.contains('EOS') & master.PowderType.isna()
master.loc[eos, 'PowderType'] = 'EOS'

# Replace instances of KM with Kennametal
km = master.PowderType.str.contains('KM') | master.MaterialType.str.contains('KM')
master.loc[km, 'PowderType'] = 'Kennametal'

master.loc[master.PowderType.str.contains('Elem'), 'PowderType'] = 'Elementum 3D'
master.loc[master.PowderType.str.contains('Carpenter'), 'PowderType'] = 'Carpenter'

# Manually validated that sieving condition is represented properly, no change

# Make all 316L instances 316
master.loc[master.MaterialType.str.contains('316L'), 'MaterialType'] = '316L'

# Correct inst where extra info given
master.MaterialType.compare(master.PowderType)

,self,other
Build ID:,,
B001,316L,EOS
B002,316L,EOS
B003,316L,EOS
B004,316L,EOS
B005,316L,EOS
B006,316L,EOS
B007,316L,EOS
B008,316L,EOS
B009,316L,EOS


In [56]:
print('Successful')
print('Before', master.Successful.unique())

success = master.Successful

master.loc[success.str.contains('no', case=False)&~success.isna(), 'Successful'] = 'no'
master.loc[success.str.contains('yes', case=False)&~success.isna(), 'Successful'] = 'yes'

print('After', master.Successful.unique())


Successful
Before ['Yes' 'No' nan 'YES!' 'YES' 'NO :(']
After ['yes' 'no' nan]


In [57]:
master.to_excel('clean_BuildDocumentation.xlsx')